In [11]:
import pandas as pd
import re
import os
import requests

#pip install natasha
from natasha import DatesExtractor, MorphVocab, AddrExtractor, MoneyExtractor
from datetime import date

#pip install pymystem3
from pymystem3 import Mystem

In [13]:
def data_load_from_file(path: str) -> pd.DataFrame:
    parser_df = pd.read_csv(path)
    expected_columns = ['name', 'salary', 'experience', 'employment_type', 'company_name',
       'adress', 'text', 'skills', 'uid', 'datatime', 'quary_name', 'link']
    if all([x in parser_df.columns for x in expected_columns]):
        parser_df = parser_df.rename(mapper={'quary_name':'query_name', 'datatime':'datetime', 'adress':'address'}, axis='columns')
        return parser_df
    else:
        return None

def data_load_from_folder(directory: str, remove_files: bool = False) -> pd.DataFrame:
    df = None
    for filename in os.listdir(directory):
        if not filename.lower().endswith('.csv'):
            continue
        f = os.path.join(directory, filename)
        if os.path.isfile(f):
            df_x = data_load_from_file(f)
            if df_x is None:
                continue
            elif df is None:
                df = df_x
            else:
                df = pd.concat([df, df_x])
            if remove_files:
                os.remove(f)

    return df

parser_df = data_load_from_folder('data/hh_parsed_folder/')
parser_df.shape

(6639, 12)

In [14]:
def data_cleaning(df: pd.DataFrame) -> pd.DataFrame:
    """clean df: dropduplicates and some NaN values"""
    df = df.drop_duplicates()
    df = df[df.skills != '[]']
    df = df.dropna(subset=['name', 'text', 'query_name', 'skills'])
    df = df.reset_index(drop=True)
    df.text = df.text.apply(lambda x: x.strip())
    return df

df = data_cleaning(parser_df)
df.shape

(5332, 12)

In [15]:
def get_currency_rates(curr=['USD', 'EUR']):
    try:
        data = requests.get('https://www.cbr-xml-daily.ru/daily_json.js').json()
        result = {}
        for c in curr:
            result[c] = data['Valute'][c]['Value']
        return result
    except:
        #! some log
        return {'USD':80.0, 'EUR':90.0}

In [16]:
def data_feature_extrator(df: pd.DataFrame) -> pd.DataFrame:
    """Extract and decode features (! copy of dataframe is not created !)
        - employment_day - 'Полная занятость', 'Частичная занятость', etc
        - employment_workhours - 'Полный день', 'Гибкий график', etc
        - publish_date - data of publishing
        - city
        - min_salary
        - max_salary
        """
    
    df['employment_day'] = df.employment_type.apply(\
        lambda x: x.split(',')[0].strip() if x is not None and len(x) > 0 else None)

    df['employment_workhours'] = df.employment_type.apply(\
        lambda x: x.split(',')[1].strip().capitalize() if x is not None and ',' in x  else None)

    morph_vocab = MorphVocab()
    extractor = DatesExtractor(morph_vocab)
    def extract_publish_date(s):
        if s != s:
            return None
        matches = extractor(s)
        match = next(matches, None)
        return date(match.fact.year, match.fact.month, match.fact.day) if match is not None else None
    df['publish_date'] = df['datetime'].apply(extract_publish_date)

    m = Mystem()
    extractor = AddrExtractor(morph_vocab)
    def extract_city(s):
        if s != s:
            return None
        matches = extractor(s)
        match = next(matches, None)
        return m.lemmatize(match.fact.value)[0].capitalize() if match is not None else None
    df['city'] = df.datetime.apply(extract_city)

    currency_rates = get_currency_rates()
    r = re.compile(r'\d+')
    default_tax = 0.13

    def extract_min_price(s):
        match = re.findall(r, s.replace(' ', '').replace(u'\xa0', ''))
        tax = default_tax if s.find('до вычета налогов') > 0 else 0
        exchange_rate = 1.0
        for k, v in currency_rates.items():
            if k in s:
                exchange_rate = v
        if len(match) == 0:
            return None
        else:
            return int(match[0]) * (1-tax) * exchange_rate

    def extract_max_price(s):
        match = re.findall(r, s.replace(' ', '').replace(u'\xa0', ''))
        tax = default_tax if s.find('до вычета налогов') > 0 else 0
        exchange_rate = 1.0
        for k, v in currency_rates.items():
            if k in s:
                exchange_rate = v
        if len(match) == 0:
            return None
        elif len(match) == 1:
            return int(match[0]) * (1-tax) * exchange_rate
        else:
            return int(match[1]) * (1-tax) * exchange_rate

    df['min_salary'] = df.salary.apply(extract_min_price)
    df['max_salary'] = df.salary.apply(extract_max_price)

    return df


In [17]:
%%time
df = data_feature_extrator(df)
df.head()

CPU times: user 5min 24s, sys: 3.5 s, total: 5min 28s
Wall time: 5min 38s


,name,salary,experience,employment_type,company_name,address,text,skills,uid,datetime,query_name,link,employment_day,employment_workhours,publish_date,city,min_salary,max_salary
0,Главный бухгалтер,от 150 000 до 170 000 руб. до вычета налогов,3–6 лет,"Полная занятость, полный день",ООО РК,"Москва, Золотая улица, 11",Обязанности: -Ведение бухгалтерского и налогов...,"['Ответственность', 'Пользователь ПК', 'Ориент...",70007635,Вакансия опубликована 20 сентября 2022 в Москве,Главный бухгалтер,https://hh.ru/vacancy/70007635?from=vacancy_se...,Полная занятость,Полный день,2022-09-20,Москва,130500.0,147900.0
1,Главный бухгалтер/Бухгалтер в единственном лице,от 120 000 до 120 000 руб. на руки,более 6 лет,"Полная занятость, полный день",ООО СТИЛОТ,"Москва, Профсоюзная улица, 93к4",Обязанности: Ведение всех участков бухучета (...,"['Бухгалтерская отчетность', 'Банк-клиент', 'Н...",69672027,Вакансия опубликована 21 сентября 2022 в Москве,Главный бухгалтер,https://hh.ru/vacancy/69672027?from=vacancy_se...,Полная занятость,Полный день,2022-09-21,Москва,120000.0,120000.0
2,Бухгалтер в единственном лице,от 100 000 руб. до вычета налогов,более 6 лет,"Полная занятость, полный день",ООО ПКФ Гудвин,NaN,Обязанности: Ведение нескольких юридических л...,"['Английский\xa0— A1 — Начальный', 'Бухгалтерс...",70085292,Вакансия опубликована 21 сентября 2022 в Санкт...,Главный бухгалтер,https://hh.ru/vacancy/70085292?from=vacancy_se...,Полная занятость,Полный день,2022-09-21,Санкт-петербург,87000.0,87000.0
3,Главный бухгалтер в производственную компанию,от 100 000 до 175 000 руб. на руки,3–6 лет,"Полная занятость, полный день",FinHelp,NaN,ВАКАНСИЯ НА ДОЛЖНОСТЬ ГЛАВНОГО БУХГАЛТЕРА В пр...,"['Налоговая отчетность', 'Бухгалтерская отчетн...",70152857,Вакансия опубликована 22 сентября 2022 в Серпу...,Главный бухгалтер,https://hh.ru/vacancy/70152857?from=vacancy_se...,Полная занятость,Полный день,2022-09-22,Серпухов,100000.0,175000.0
4,Старший бухгалтер / Заместитель главного бухга...,от 85 000 до 95 000 руб. до вычета налогов,3–6 лет,"Полная занятость, полный день",ООО Бумажный клуб,"Москва, Щукинская, Габричевского улица, 5К1","Белой, законопослушной компании-участнику ВЭД,...","['Банк-клиент', '1С: Предприятие 8', 'Налогова...",69126996,Вакансия опубликована 21 сентября 2022 в Москве,Главный бухгалтер,https://hh.ru/vacancy/69126996?from=vacancy_se...,Полная занятость,Полный день,2022-09-21,Москва,73950.0,82650.0


In [18]:
def drop_text_duplicates(df: pd.DataFrame) -> pd.DataFrame:
    # если id-ники вакансий разные, а текст одинаковый, 
    # то последенюю по дате публикации, а если даты одинаковые то по более популярному городу
    cities = df['city'].value_counts(normalize=True, dropna=False)
    df['city_rating'] = df.city.apply(lambda x: cities[x])
    return df.sort_values(['text', 'publish_date', 'city_rating'], ascending=[True, False, False]) \
        .groupby('text', as_index=False).first()

df = drop_text_duplicates(df)
print(df.shape)

(4208, 19)


In [19]:
def save_data_to_csv(df: pd.DataFrame, path: str):
    df.to_csv(path, index=False)

save_data_to_csv(df, 'data/all_vacancies.csv')